### Symbolic treatment of the fluid equations

In this notebook we obtain the equations, and all relevant quantities starting from the generating function and in arbitrary dimensions. 
We spacialize for the conformal case, but this is not needed. 

We use a flat, constant metric, but it is not difficult to add an arbitrary metric latter on, even curvature.

In [1]:
using Symbolics
using LinearAlgebra
using Latexify
using Plots

The variables are going to be a vector-tensor $\zeta_A = (\zeta_a, \zeta_{ab})$ with the tensor part being symmetric and (latter) trace free. We are going to use both representations, either a big vector or a pair vector-tensor.
With functions going from one to the other.

We first define some tensors and take a symmetrize them using the upper triangular part as the relevant tensor part, contrary to take the average of the components

In [2]:
D = Int64(4) #dimensions
ζ = Vector{Float64}(undef,D+D*(D+1)÷2)
ζ_v = Vector{Float64}(undef,D)
ζ_t = Array{Float64}(undef,D,D)

function make_g(D)
    u = ones(D)
    u[1] = -1.0
    g = Diagonal(u) + zeros(D,D)
end

g = make_g(4)
latexify(g)
#typeof(g)


L"\begin{equation}
\left[
\begin{array}{cccc}
-1.0 & 0.0 & 0.0 & 0.0 \\
0.0 & 1.0 & 0.0 & 0.0 \\
0.0 & 0.0 & 1.0 & 0.0 \\
0.0 & 0.0 & 0.0 & 1.0 \\
\end{array}
\right]
\end{equation}
"

In [4]:
function vector2symmat!(v,sm)
    M = length(sm[1,:]) # N = M*(M+1)/2 
    N = length(v)
    if M*(M+1)÷2 != N
        error("Dimension missmatch, M = $M, N= $N")
    end
    L = 1
    for i in 1:M
        for j in i:M
            sm[i,j] = v[L]
            sm[j,i] = sm[i,j]
            L=L+1
        end
    end
end

v = [1;2;3;4;5;6]
sm = zeros(3,3)

vector2symmat!(v,sm)

sm

3×3 Matrix{Float64}:
 1.0  2.0  3.0
 2.0  4.0  5.0
 3.0  5.0  6.0

In [5]:
function symmat2vector(sm)
    M = length(sm[1,:])
    N = M*(M+1)÷2
    v = zeros(N)
    l = 1
    for i in 1:M
        for j in i:M
            v[l] = sm[i,j]
            l = l+1
        end
    end
    return v
end

symmat2vector(sm)

    

6-element Vector{Float64}:
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0

In [6]:
function vector2symmat(v)
    
    N = length(v)
    MM = (-1 + sqrt(1+8N))/2 
    if MM % 1 ≈ 0
        M = Int64(MM)
    else
        error("the vector dimension does not corresponds to the number of free entries of a symmetric matrix")
    end
    #println("M = $M")
    sm = zeros(M,M)
    
    L = 1
    for i in 1:M
        for j in i:M
            sm[i,j] = v[L]
            sm[j,i] = sm[i,j]
            L=L+1
        end
    end
    return sm
end

v = [1;2;3;4;5;6;7;8;9;10]


sm = vector2symmat(v)

4×4 Matrix{Float64}:
 1.0  2.0  3.0   4.0
 2.0  5.0  6.0   7.0
 3.0  6.0  8.0   9.0
 4.0  7.0  9.0  10.0

In [116]:
ζ_t = [-1. 2 3 4;
        0 1 5 6;
        4 0 -1 8;
        0 0 0 -1]
println(tr(ζ_t'*g'*g*ζ_t))

function my_symmetrization!(s)
        M = length(s[:,1])
        for i ∈ 1:M
                for j ∈ i:M
                        s[j,i] = s[i,j]
                end
        end
end

function my_symmetrization(s)
        ss = copy(s)
        M = length(s[:,1])
        for i ∈ 1:M
                for j ∈ i:M
                        ss[j,i] = s[i,j]
                end
        end
        return ss[:,:]
end

ζ_v = [-1.; 0.1; 0.2; 0.3]
#ζ_v = [0.; 0.; 0.; 0.]

my_symmetrization!(ζ_t)
latexify(ζ_t)

ζ = [ζ_v; symmat2vector(ζ_t)]

174.0


14-element Vector{Float64}:
 -1.0
  0.1
  0.2
  0.3
 -1.0
  2.0
  3.0
  4.0
  1.0
  5.0
  6.0
 -1.0
  8.0
 -1.0

Likewise we define functions which take either a long vector or the pair vector-tensor

In [118]:
#function Φ₁(ζ_v::Array{Float64,1}, ζ_s::Array{Float64,2}, p::Array{Float64,1})
function Φ_vt(ζ_v, ζ_s, p)
    D = length(ζ_v)
    g = make_g(D)
    #println(g)
    χ₀ = p[1]
    χ₁ = p[2]
    χ₂ = p[3]
    #ζ_s = my_symmetrization!(Symbolics.scalarize(ζ_t))
    μ = Symbolics.scalarize(ζ_v)'*g*Symbolics.scalarize(ζ_v)
    ν = (g*Symbolics.scalarize(ζ_v))'*Symbolics.scalarize(ζ_s)*Symbolics.scalarize(g*ζ_v)
    l = Symbolics.scalarize(ζ_s)*Symbolics.scalarize(g*ζ_v)
    𝚶 = l'* g * l
    τ₂= tr(Symbolics.scalarize(ζ_s)*g'*Symbolics.scalarize(ζ_s)*g)
    D1 = D//2+1
    D2 = D//2+2
    return χ₀*μ^(1-D//2) + χ₁*ν*μ^(-1-D//2) + χ₂*(τ₂ -4*D1*𝚶*μ^(-1) + 2*D1*D2*ν^2*μ^(-2)) # no es la función correcta, solo probando
    #return 𝚶
end

function Φ_v(ζ,p)
    ζ_t = vector2symmat(ζ[D+1:end])
    return Φ_vt(ζ[1:D], ζ_t, p)
end

Φ_vt(ζ_v,ζ_t, [1.,1.,1.])#, ζ_t, [1.;1;1])
Φ_v(ζ, [1.,1.,1.])
#ζ_v
#ζ_t

2076.290590765593

In [119]:
@variables zv[1:D], zs[1:D,1:D], pv[1:3]

Φ_vt(zv, zs, pv)

((zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2))^(-1//1))*pv[1] + (((-12//1)*((zv[2]*zs[2, 2] + zv[3]*zs[2, 3] + zv[4]*zs[2, 4] - zv[1]*zs[2, 1])^2) - (12//1)*((zv[2]*zs[3, 2] + zv[3]*zs[3, 3] + zv[4]*zs[3, 4] - zv[1]*zs[3, 1])^2) - (12//1)*((zv[2]*zs[4, 2] + zv[3]*zs[4, 3] + zv[4]*zs[4, 4] - zv[1]*zs[4, 1])^2) - (12//1)*(zv[2]*zs[1, 2] + zv[3]*zs[1, 3] + zv[4]*zs[1, 4] - zv[1]*zs[1, 1])*(zv[1]*zs[1, 1] - zv[2]*zs[1, 2] - zv[3]*zs[1, 3] - zv[4]*zs[1, 4])) / (zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2)) + zs[1, 1]^2 + zs[2, 2]^2 + zs[3, 3]^2 + zs[4, 4]^2 + (24//1)*(((zv[2]*zs[2, 2] + zv[3]*zs[3, 2] + zv[4]*zs[4, 2] - zv[1]*zs[1, 2])*zv[2] + (zv[2]*zs[2, 3] + zv[3]*zs[3, 3] + zv[4]*zs[4, 3] - zv[1]*zs[1, 3])*zv[3] + (zv[2]*zs[2, 4] + zv[3]*zs[3, 4] + zv[4]*zs[4, 4] - zv[1]*zs[1, 4])*zv[4] - (zv[2]*zs[2, 1] + zv[3]*zs[3, 1] + zv[4]*zs[4, 1] - zv[1]*zs[1, 1])*zv[1])^2)*((1 / (zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2)))^2) + 2zs[2, 3]*zs[3, 2] + 2zs[2, 4]*zs[4, 2] + 2zs[3, 4]*zs[4, 3] - 2.0zs[1, 2]*zs[

In [120]:
zvt = [zv[i] for i ∈ 1:D]
JΦ_v = Symbolics.gradient(Φ_vt(zv, zs, pv), zvt)

4-element Vector{Num}:
 (((12.0zv[1]*zs[1, 1] - (12//1)*zv[2]*zs[1, 2] - (12//1)*zv[3]*zs[1, 3] - (12//1)*zv[4]*zs[1, 4])*zs[1, 1] + 12.0(2zv[2]*zs[2, 2] + 2zv[3]*zs[2, 3] + 2zv[4]*zs[2, 4] - 2.0zv[1]*zs[2, 1])*zs[2, 1] + 12.0(2zv[2]*zs[3, 2] + 2zv[3]*zs[3, 3] + 2zv[4]*zs[3, 4] - 2.0zv[1]*zs[3, 1])*zs[3, 1] + 12.0(2zv[2]*zs[4, 2] + 2zv[3]*zs[4, 3] + 2zv[4]*zs[4, 4] - 2.0zv[1]*zs[4, 1])*zs[4, 1] - (12.0zv[2]*zs[1, 2] + 12.0zv[3]*zs[1, 3] + 12.0zv[4]*zs[1, 4] - 12.0zv[1]*zs[1, 1])*zs[1, 1]) / (zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2)) + (96.0(((zv[2]*zs[2, 2] + zv[3]*zs[3, 2] + zv[4]*zs[4, 2] - zv[1]*zs[1, 2])*zv[2] + (zv[2]*zs[2, 3] + zv[3]*zs[3, 3] + zv[4]*zs[4, 3] - zv[1]*zs[1, 3])*zv[3] + (zv[2]*zs[2, 4] + zv[3]*zs[3, 4] + zv[4]*zs[4, 4] - zv[1]*zs[1, 4])*zv[4] - (zv[2]*zs[2, 1] + zv[3]*zs[3, 1] + zv[4]*zs[4, 1] - zv[1]*zs[1, 1])*zv[1])^2)*(1 / ((zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2))^2))*zv[1]) / (zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2)) + 2.0(((-12//1)*((zv[2]*zs[2, 2] + zv[3]*zs[

In [121]:
JΦ_v_exp = Symbolics.build_function(JΦ_v,zv, zs, pv);
JΦ_ve = eval(JΦ_v_exp[1]);
JΦ_ve(ζ_v,ζ_t, pv)

4-element Vector{Num}:
 3844.2350987963346pv[3] - 2.7041644131963225pv[1] - 45.11815692109603pv[2]
 5247.928106959135pv[3] - 0.2704164413196323pv[1] - 20.233701815344034pv[2]
 6184.033607103778pv[3] - 0.5408328826392645pv[1] - 27.57545700963584pv[2]
 6942.1185555988795pv[3] - 0.8112493239588967pv[1] - 33.03058586913951pv[2]

In [122]:
zst = vec([zs[i,j] for i ∈ 1:D, j ∈ 1:D])
JΦ_s = reshape(Symbolics.gradient(Φ_vt(zv, zs, pv), zst), (D,D)) 

4×4 Matrix{Num}:
 (((12.0zv[1]*zs[1, 1] - (12//1)*zv[2]*zs[1, 2] - (12//1)*zv[3]*zs[1, 3] - (12//1)*zv[4]*zs[1, 4])*zv[1] - (12.0zv[2]*zs[1, 2] + 12.0zv[3]*zs[1, 3] + 12.0zv[4]*zs[1, 4] - 12.0zv[1]*zs[1, 1])*zv[1]) / (zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2)) + 2zs[1, 1] + (24//1)*(2(zv[2]*zs[2, 2] + zv[3]*zs[3, 2] + zv[4]*zs[4, 2] - zv[1]*zs[1, 2])*zv[2] + 2(zv[2]*zs[2, 3] + zv[3]*zs[3, 3] + zv[4]*zs[4, 3] - zv[1]*zs[1, 3])*zv[3] + 2(zv[2]*zs[2, 4] + zv[3]*zs[3, 4] + zv[4]*zs[4, 4] - zv[1]*zs[1, 4])*zv[4] - 2.0(zv[2]*zs[2, 1] + zv[3]*zs[3, 1] + zv[4]*zs[4, 1] - zv[1]*zs[1, 1])*zv[1])*((1 / (zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2)))^2)*(zv[1]^2))*pv[3] + ((zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2))^(-3//1))*(zv[1]^2)*pv[2]  …                                                                (((12.0zv[2]*zs[1, 2] + 12.0zv[3]*zs[1, 3] + 12.0zv[4]*zs[1, 4] - 12.0zv[1]*zs[1, 1])*zv[4] - (12.0zv[1]*zs[1, 1] - (12//1)*zv[2]*zs[1, 2] - (12//1)*zv[3]*zs[1, 3] - (12//1)*zv[4]*zs[1, 4])*zv[4]) / (zv[2

In [123]:
JΦ_s = (JΦ_s + JΦ_s')/2

4×4 Matrix{Num}:
        (((12.0zv[1]*zs[1, 1] - (12//1)*zv[2]*zs[1, 2] - (12//1)*zv[3]*zs[1, 3] - (12//1)*zv[4]*zs[1, 4])*zv[1] - (12.0zv[2]*zs[1, 2] + 12.0zv[3]*zs[1, 3] + 12.0zv[4]*zs[1, 4] - 12.0zv[1]*zs[1, 1])*zv[1]) / (zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2)) + 2zs[1, 1] + (24//1)*(2(zv[2]*zs[2, 2] + zv[3]*zs[3, 2] + zv[4]*zs[4, 2] - zv[1]*zs[1, 2])*zv[2] + 2(zv[2]*zs[2, 3] + zv[3]*zs[3, 3] + zv[4]*zs[4, 3] - zv[1]*zs[1, 3])*zv[3] + 2(zv[2]*zs[2, 4] + zv[3]*zs[3, 4] + zv[4]*zs[4, 4] - zv[1]*zs[1, 4])*zv[4] - 2.0(zv[2]*zs[2, 1] + zv[3]*zs[3, 1] + zv[4]*zs[4, 1] - zv[1]*zs[1, 1])*zv[1])*((1 / (zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2)))^2)*(zv[1]^2))*pv[3] + ((zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2))^(-3//1))*(zv[1]^2)*pv[2]                                                                                                                                                                                                                                                                        

In [124]:
JΦ_s[2,1]

(1//2)*((12.0(2zv[2]*zs[2, 2] + 2zv[3]*zs[2, 3] + 2zv[4]*zs[2, 4] - 2.0zv[1]*zs[2, 1])*zv[1]) / (zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2)) - 2.0zs[1, 2] - 24.0(2(zv[2]*zs[2, 2] + zv[3]*zs[3, 2] + zv[4]*zs[4, 2] - zv[1]*zs[1, 2])*zv[2] + 2(zv[2]*zs[2, 3] + zv[3]*zs[3, 3] + zv[4]*zs[4, 3] - zv[1]*zs[1, 3])*zv[3] + 2(zv[2]*zs[2, 4] + zv[3]*zs[3, 4] + zv[4]*zs[4, 4] - zv[1]*zs[1, 4])*zv[4] - 2.0(zv[2]*zs[2, 1] + zv[3]*zs[3, 1] + zv[4]*zs[4, 1] - zv[1]*zs[1, 1])*zv[1])*((1 / (zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2)))^2)*zv[1]*zv[2])*pv[3] + (1//2)*(((12.0zv[2]*zs[1, 2] + 12.0zv[3]*zs[1, 3] + 12.0zv[4]*zs[1, 4] - 12.0zv[1]*zs[1, 1])*zv[2] - (12.0zv[1]*zs[1, 1] - (12//1)*zv[2]*zs[1, 2] - (12//1)*zv[3]*zs[1, 3] - (12//1)*zv[4]*zs[1, 4])*zv[2]) / (zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2)) - 2.0zs[2, 1] - 24.0(2(zv[2]*zs[2, 2] + zv[3]*zs[3, 2] + zv[4]*zs[4, 2] - zv[1]*zs[1, 2])*zv[2] + 2(zv[2]*zs[2, 3] + zv[3]*zs[3, 3] + zv[4]*zs[4, 3] - zv[1]*zs[1, 3])*zv[3] + 2(zv[2]*zs[2, 4] + zv[3]*zs[3, 4] + 

In [125]:
JΦ_s_exp = Symbolics.build_function(JΦ_s, zv, zs, pv);
JΦ_se = eval(JΦ_s_exp[1]);
JΦ_se(ζ_v, ζ_t, pv)

4×4 Matrix{Num}:
 255.653pv[3] - 1.57219pv[2]    …  155.807pv[3] - 0.471657pv[2]
  91.5327pv[3] - 0.157219pv[2]      49.311pv[3] - 0.0471657pv[2]
 127.856pv[3] - 0.314438pv[2]       73.4592pv[3] - 0.0943313pv[2]
 155.807pv[3] - 0.471657pv[2]       73.0957pv[3] - 0.141497pv[2]

In [126]:
T_s = Symbolics.jacobian(JΦ_ve(zv, zs, pv), zvt)

4×4 Matrix{Num}:
 ((96.0(((zv[1]*zs[1, 1] - zv[2]*zs[2, 1] - zv[3]*zs[3, 1] - zv[4]*zs[4, 1])*zv[1] + (zv[2]*zs[2, 2] + zv[3]*zs[3, 2] + zv[4]*zs[4, 2] - zv[1]*zs[1, 2])*zv[2] + (zv[2]*zs[2, 3] + zv[3]*zs[3, 3] + zv[4]*zs[4, 3] - zv[1]*zs[1, 3])*zv[3] + (zv[2]*zs[2, 4] + zv[3]*zs[3, 4] + zv[4]*zs[4, 4] - zv[1]*zs[1, 4])*zv[4])^2) + 96.0(2(zv[1]*zs[1, 1] - zv[2]*zs[2, 1] - zv[3]*zs[3, 1] - zv[4]*zs[4, 1])*zv[1] + 2(zv[2]*zs[2, 2] + zv[3]*zs[3, 2] + zv[4]*zs[4, 2] - zv[1]*zs[1, 2])*zv[2] + 2(zv[2]*zs[2, 3] + zv[3]*zs[3, 3] + zv[4]*zs[4, 3] - zv[1]*zs[1, 3])*zv[3] + 2(zv[2]*zs[2, 4] + zv[3]*zs[3, 4] + zv[4]*zs[4, 4] - zv[1]*zs[1, 4])*zv[4])*(2.0zv[1]*zs[1, 1] - zv[2]*zs[1, 2] - zv[2]*zs[2, 1] - zv[3]*zs[1, 3] - zv[3]*zs[3, 1] - zv[4]*zs[1, 4] - zv[4]*zs[4, 1])*zv[1]) / ((zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2))^3) + (24.0(zs[1, 1]^2) - 24.0(zs[2, 1]^2) - 24.0(zs[3, 1]^2) - 24.0(zs[4, 1]^2)) / (zv[2]^2 + zv[3]^2 + zv[4]^2 - (zv[1]^2)) + (((24.0zv[1]*zs[1, 1] - 24.0zv[2]*zs[1, 2] - 24.0zv[3

In [127]:
T_exp = Symbolics.build_function(T_s, zv, zs, pv);
T = eval(T_exp[1]);
tr(g*T(ζ_v, ζ_t, [1.0;1.0;0.0]))

-2.842170943040401e-14

4×4 Matrix{Float64}:
 -1.0  2.0  3.0   4.0
  2.0  1.0  5.0   6.0
  3.0  5.0  1.0   8.0
  4.0  6.0  8.0  -1.0

In [13]:
function Φ₂(ζᵥ, ζᵥᵥ, p, g)
    χ₀,χ₁,χ₂ = p
    #ζₕ = vector2symmat(ζᵥᵥ)
    μ = ζᵥ'*g*ζᵥ
    #ν = (g*ζᵥ)'*ζₕ*(g*ζᵥ)
    #τ = tr(ζₕ*g'*ζₕ*g)
    return χ₀*μ #+ χ₁*ν + χ₂*ν*τ
end

Φ₂([-1; 0.5; 0; 0], [1;2;3;4;5;6;7;8;9;10], (1, 2, 3), g)

-0.75

In [14]:
#@variables ζ[1:D], ζζ[1:D*(D+1)÷2], p[1:3]
#@variables zv[1:D], zs[1:D*(D+1)÷2], pv[1:3], gg[1:D,1:D]
@variables zv[1:4], zs[1:10], pv[1:3]#, gg[1:4,1:4]
Jζ = Symbolics.jacobian(Φ₂(zv,zs,pv,g),zv) 

LoadError: MethodError: no method matching jacobian(::Num, ::Symbolics.Arr{Num, 1})
[0mClosest candidates are:
[0m  jacobian([91m::Union{SymbolicUtils.Symbolic{AbstractVector{T}}, Symbolics.Arr{T, 1}, Symbolics.ArrayOp{AbstractVector{T}}, SymbolicUtils.Term{Symbolics.Arr{T, 1}, M} where M}[39m, ::Union{SymbolicUtils.Symbolic{AbstractVector{T}}, Symbolics.Arr{T, 1}, Symbolics.ArrayOp{AbstractVector{T}}, SymbolicUtils.Term{Symbolics.Arr{T, 1}, M} where M}; simplify) where T at /Users/reula/.julia/packages/Symbolics/ohRLi/src/diff.jl:372
[0m  jacobian([91m::AbstractVector{T} where T[39m, ::AbstractVector{T} where T; simplify) at /Users/reula/.julia/packages/Symbolics/ohRLi/src/diff.jl:368

In [43]:
l = Symbolics.scalarize(zs)*Symbolics.scalarize(g*zv)
𝚶 = l'* g * l
simplify(𝚶)

(zv[2]*zs[1, 2] + zv[3]*zs[1, 3] + zv[4]*zs[1, 4] - zv[1]*zs[1, 1])*(zv[1]*zs[1, 1] - zv[2]*zs[1, 2] - zv[3]*zs[1, 3] - zv[4]*zs[1, 4]) + (zv[2]*zs[2, 2] + zv[3]*zs[2, 3] + zv[4]*zs[2, 4] - zv[1]*zs[2, 1])^2 + (zv[2]*zs[3, 2] + zv[3]*zs[3, 3] + zv[4]*zs[3, 4] - zv[1]*zs[3, 1])^2 + (zv[2]*zs[4, 2] + zv[3]*zs[4, 3] + zv[4]*zs[4, 4] - zv[1]*zs[4, 1])^2

In [60]:
reshape(Symbolics.gradient(𝚶, zst, simplify=true), (D,D))'[1,2]

-(2zv[2]*zs[2, 2] + 2zv[3]*zs[2, 3] + 2zv[4]*zs[2, 4] - 2.0zv[1]*zs[2, 1])*zv[1]

In [61]:
reshape(Symbolics.gradient(𝚶, zst, simplify=true), (D,D))'[2,1]

(zv[1]*zs[1, 1] - zv[2]*zs[1, 2] - zv[3]*zs[1, 3] - zv[4]*zs[1, 4])*zv[2] - (zv[2]*zs[1, 2] + zv[3]*zs[1, 3] + zv[4]*zs[1, 4] - zv[1]*zs[1, 1])*zv[2]